In [1]:
from src.Model import Model
import torch
import torch.nn.functional as F
from tqdm import tqdm
from src.Tools import generate_text,get_eos_embedding,update_sequence_mask,get_one_hot
from src.train import one_epoch_caption
from src.Dataset import QADataset,CaptionDataset
from torch.utils.data import DataLoader

model = Model()

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['embeddings.mask_token']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
QA_dataset = QADataset('Data/annotations/QA/train.json')
Caption_dataset = CaptionDataset('Data/annotations/Descriptive/train.json')
data_loader = DataLoader(Caption_dataset, batch_size= 32, shuffle = True)

loading annotations into memory...
Done (t=0.15s)
creating index...
index created! imgs = 23431, anns = 100575


In [3]:
def one_epoch_QA(model,dataloader,optimizer):
    model.train()
    train_loss = 0.0
    model = model.to('cuda')
    print('cuda' if next(model.parameters()).is_cuda else 'cpu')

    for batch_idx, ((image,q_ids),text_ids) in enumerate(tqdm(dataloader, total=len(dataloader), ncols=60)):
        optimizer.zero_grad()
        print(image.size())
        eos_embedding,eos_attention_mask = get_eos_embedding(model,image.size(0))
        input_sequence,attention_mask = model.get_sequence(image, q_ids)

        input_sequence = torch.cat((input_sequence, eos_embedding), dim=1).to('cuda')
        print(attention_mask.device,eos_attention_mask.device)
        attention_mask = torch.cat((attention_mask, eos_attention_mask), dim=1).to('cuda')
        loss = 0

        for i in range(2):
            probablities = model(input_sequence,attention_mask)
            input_sequence,attention_mask = update_sequence_mask(model,probablities,input_sequence,attention_mask)
            out_encoding = get_one_hot(text_ids,len(model.generator_tokenizer.vocab),i)
            loss += F.cross_entropy(probablities,out_encoding)

        train_loss += loss

        loss.backward()

        optimizer.step()
    print(f'{batch_idx}:{train_loss/len(dataloader)}')

    return train_loss

In [4]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss = one_epoch_caption(model,data_loader,optimizer)

cuda


  0%|                              | 0/3143 [00:00<?, ?it/s]

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
  0%|                              | 0/3143 [00:02<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument tensors in method wrapper_CUDA_cat)